# Project Assistance: Analyzing Customer Churn

# 1) Importing Data

In [ ]:
import sqlalchemy as sa
import pandas as pd
engine = sa.create_engine('sqlite:///telco_churn.db')
connection = engine.connect()
sql_query = "SELECT * FROM churn_data JOIN cities ON cities.area_code = churn_data.local_area_code"
df = pd.read_sql(sql_query, connection)
df.head()
df.dtypes
connection.close()


# 2) Check and clean the data

In [ ]:
# Drop double columns
df = df.drop('local_area_code',axis=1)

In [ ]:
# Check the missing values
import numpy as np
# df.isna().sum()

df = df.dropna(axis=0, how='any')
df.isna().sum()

# df.dropna?

In [ ]:

# Changing the datatypes 
df.dtypes
has_to_be_int_list = [0,3,5,8,11,13]
has_to_be_categorical_list = [1,2,14,15,16,17]
for i in has_to_be_int_list:
    df.iloc[:,i] = df.iloc[:,i].astype('int')
for i in has_to_be_categorical_list:
    df.iloc[:,i] = df.iloc[:,i].astype('category')
df.dtypes
# Check for wrong values

df.describe()

In [ ]:
mask = (df.loc[:,'customer_service_calls'] >= 0)
# df = df.drop(df.index[mask])
df = df.loc[mask,:]
df.describe()

# 3) What are the names of the four cities with the highest rates of customer churn?

In [ ]:
# What is the overall churn ratio in the dataset?
df_churn_rate = pd.crosstab(index=df.loc[:,'churn'],columns='count',normalize='columns')

pd.options.display.float_format = '{:,.2f}'.format
df_churn_rate

In [ ]:
# What is the churn ratio based on the city?
churn_mask = (df.loc[:,'churn'] == 1)

df_city = pd.crosstab(index=df.loc[:,'city'], columns=df.loc[churn_mask,'churn'])
df_city.columns=['count']
df_city = df_city.sort_values(by='count', ascending=False)
print(df_city)

# pd.crosstab(index=df.loc[:,'city'], columns=df.loc[:,'churn'],normalize='all')

biggest_cities_left = df_city.iloc[:4,0]
list_cities = biggest_cities_left.index.tolist()
list_cities


In [ ]:
# Is the number of datapoints for every city reasonable?
Jacksonville_mask = (df.loc[:,'city']=='Jacksonville') 
Orlando1_mask = (df.loc[:,'city']=='Orlando1') 
Cape_Coral_mask = (df.loc[:,'city']=='Cape Coral') 
Orlando2_mask = (df.loc[:,'city']=='Orlando2')


jacksonville_count = df.loc[Jacksonville_mask,:].shape
print(jacksonville_count)

orlando1_count = df.loc[Orlando1_mask,:].shape
print(orlando1_count)

cape_coral_count = df.loc[Cape_Coral_mask,:].shape
print(cape_coral_count)

orlando2_count = df.loc[Orlando2_mask,:].shape
print(orlando2_count)

df.shape

# 4a) Which categorical data series should be used to identify customers who will possibly leave soon? 

# Which customers should be contacted based on this data series?

In [ ]:
# Check 'international_plan'
df_international_plan = pd.crosstab(columns = df.loc[:,'international_plan'],
                                     index= df.loc[:,'churn'],normalize='columns')
df_international_plan

In [ ]:
# Check 'voice_mail_plan'
pd.crosstab(index = df.loc[:,'voice_mail_plan'], columns = df.loc[:,'churn'])


In [ ]:
pd.crosstab(index = df.loc[:,'voice_mail_plan'], columns = df.loc[:,'churn'],normalize='index')


In [ ]:
# Select customers based on categorical data
customers_that_might_leave_mask = (df.loc[:,'international_plan'] == 'yes') & (df.loc[:,'churn'] == 0)
df_customers_prob_leave = df.loc[customers_that_might_leave_mask,:]
df_customers_prob_leave


# 4b) Which integer data series would you also use for this and how would you set the threshold?

# Which customers should be contacted based on this data series?

In [ ]:
# How many unique values are in each column?
cols_to_check = ['account_length', 'number_vmail_messages', 'total_day_calls', 'total_eve_calls',
                 'total_night_calls', 'customer_service_calls']

for i in cols_to_check:
    print(len(df.loc[:,i].unique().tolist()))

In [ ]:
# Check column that can be treated as categorical
#  'customer_service_calls'

df_service_calls = pd.crosstab(index=df.loc[:,'customer_service_calls'],
                               columns=df.loc[:,'churn'], normalize='index')
# pd.crosstab(index=df.loc[:,'customer_service_calls'], columns='count',normalize='columns')

%matplotlib inline
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=[6,6])
df_service_calls.plot(kind='bar',legend=False, ax=ax)
ax.xaxis.set_tick_params(labelrotation=0)

#customers that have >3 calls to the service are likely to leave because they
#                                                 did not get satisfied after calling many times

# df_service_calls.describe()
# df_service_calls

In [ ]:
# Check other integer columns with boxplots or histograms
#'account_length', 'number_vmail_messages', 'total_day_calls', 'total_eve_calls', 'total_night_calls'
list_to_create_graphs = ['account_length', 'number_vmail_messages',
                         'total_day_calls', 'total_eve_calls', 'total_night_calls']
def to_create(df,col_of_info):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[19,8])
    df.boxplot(column=col_of_info, by='churn', ax=ax[0])
    df.groupby('churn')[col_of_info].plot(kind='hist', bins = 40, legend = True, ax=ax[1])
    
#     return fig

In [ ]:
for i in list_to_create_graphs:
    to_create(df,i)

In [ ]:
# Select customers based on integer column

#so its better just to use the boolean masking from the previous cell without counting whiskers
mask = (df.loc[:,'churn'] == 0) & (df.loc[:,'customer_service_calls'] > 3)
df_temp_answer_4b = df.loc[mask,:]
df_temp_answer_4b

# 4c) Which floating point data series could you use to help with this selection?

# Determine the threshold for this using logistic regression. 

# Which customers should be contacted based on this data series?

In [ ]:
# Check ..._charge and ..._minutes columns;  Is there any kind of linear depency?

# Sollution:
# Normally, a certain amount is charged per minute. Can we leave a part out of the evaluation?
# To do this, we'll make a plot with one column on the x-axis and the other on the y-axis
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=[18, 6])  # define a plot with 3 Axes (day, eve, night)
df.plot(kind='scatter', 
        x='total_day_minutes', 
        y='total_day_charge', 
        ax=axs[0],
        xlim=(0,500),
        ylim=(0,60));  # check day columns
df.plot(kind='scatter', 
        x='total_eve_minutes', 
        y='total_eve_charge', 
        ax=axs[1],
        xlim=(0,500),
        ylim=(0,60));  # check evening columns
df.plot(kind='scatter', 
        x='total_night_minutes', 
        y='total_night_charge', 
        ax=axs[2],
        xlim=(0,500),
        ylim=(0,60));  # check night columns
# So we see that there is a linear relationship here. So we only need one of the two columns at a time.
# In the plot total_day_charge and total_day_minutes there are a few outliers, i.e. data points that cannot be explained. Let's take a closer look at these.

In [ ]:
# Identifying outliers and deal with them
mask = (df.loc[:,'total_day_minutes'] > 380)
df = df.drop(df.index[mask],axis=0)
df

In [ ]:
# Pair plot
df_temp = df.loc[:, ['total_day_minutes', 'total_day_charge', 'total_eve_minutes', 'total_eve_charge', 'total_night_minutes', 'total_night_charge']]
import seaborn as sns
sns.pairplot(df_temp);

In [ ]:
# Correlation matrix


fig, ax = plt.subplots(figsize=(10,10))         # Sample figsize in inches
# sns.heatmap(df_temp.corr(), annot=True, ax=ax);
sns.heatmap(df_temp.corr(), annot=True, ax=ax);

In [ ]:
# Logistic regression
import statsmodels.formula.api as smf
df.loc[:,'churn'] = pd.to_numeric(df.loc[:,'churn'])
# print(df.dtypes)
# Define a logit model and fit it
model = smf.logit(formula='churn ~ total_day_minutes', 
                  data=df)
result = model.fit()

# Check model summary
result.summary()

In [ ]:
# Use the trained results to predict a given range and plot the S-curve
# after experimenting with logistic regression 
# I came with a conclusion that total_day_mins have the most Pseudo R-sq. (0.05)
# as total_day_mins are float, we have to convert to int
df.loc[:,'total_day_minutes'] = df.loc[:,'total_day_minutes'].astype(int)
X = pd.Series(range(800))
X_df = pd.DataFrame(X)
X_df.columns = ['total_day_minutes']
p_y = result.predict(X_df)
p_y.plot();

In [ ]:
# Calculate treshold value and plot it as a vertical line in logistic regression plot
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots()
ax.plot(p_y)
ax.vlines(x=p_y[p_y>=0.5].index[0], ymin=0, ymax=1, colors='black');
# median of churn 1 total_day_minutes

In [ ]:
# Identify customer(s) to be contacted based on critical value from logistic regression
# p_y[p_y >= 0.5].index[0]
fig, ax = plt.subplots(figsize=[8, 8])
df.groupby('churn')['total_day_minutes'].plot(kind='hist', bins=50, ax=ax)
ax.set(xlim=[0, 800])
ax.vlines(x=349, ymin=0, ymax=160);


In [ ]:
mask = df.loc[:, 'total_day_minutes']>=p_y[p_y>=0.5].index[0]

df_that_might_leave = df.loc[mask,:]
final_mask = df.loc[:,'churn'] == 0
df_that_might_leave = df_that_might_leave.loc[final_mask,:]
df_that_might_leave

# 5) Visualizing the cities and other selected data series.

In [ ]:
# Column chart of urban areas
plt.style.use('fivethirtyeight')

colors = ['#9966CC','#483D8B']

df_city = pd.crosstab(index=df.loc[:,'city'], columns=df.loc[:,'churn'], normalize='index')

df_city = df_city.sort_values(by=1, ascending=False)

fig,ax = plt.subplots(figsize=[17,8])
df_city.plot(kind='bar',legend=True, ax=ax, color = colors)
ax.xaxis.set_tick_params(labelrotation=0)
ax.legend(['Customers that currently use the company services', 'Customers that left the company'],
          bbox_to_anchor=(0.0, 1.01), loc='upper left',prop={'size': 11});
ax.set(title='Cities with the highest rates of customer churn', xlabel='cities');

In [ ]:
# Column chart of categorical data
plt.style.use('fivethirtyeight')
df_international_plan = pd.crosstab(columns = df.loc[:,'international_plan'],
                                     index= df.loc[:,'churn'],normalize='columns')

# print(df_international_plan)

colors = ['#E0B0FF','#9370DB']

explode_=[0.05,0.05]
explode_1=[0.01,0.05]
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,6))
df_international_plan.plot(kind='pie',y='no',ax=ax[0], labels=['',''], explode=explode_, colors=colors)
df_international_plan.plot(kind='pie',y='yes',ax=ax[1], labels=['',''], explode=explode_1, legend=False,colors=colors);
fig.suptitle('Percentage of customers that use international plan', fontsize=17);
ax[0].legend(['present customers','left customers'],
             bbox_to_anchor=(0.0,1.15), loc='upper left', fontsize=13);
ax[0].set_ylabel('no international plan customers', fontsize=17)
ax[1].set_ylabel('international plan customers', fontsize=17);


t1 = ax[0].text(1.03, -0.4, '11%')
t2 = ax[0].text(-1.1, 0.7, '89%')

t3 = ax[1].text(0.7, 0.9, '57%')
t4 = ax[1].text(-1, -0.9, '43%')

In [ ]:
plt.style.use('seaborn')
colors = ['#9966CC','#D891EF']
df_service_calls = pd.crosstab(index=df.loc[:,'customer_service_calls'],
                              columns=df.loc[:,'churn'], normalize='index')
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[10,6])
df_service_calls.plot(kind='bar', legend=False, ax=ax[0],color = colors)
ax[0].xaxis.set_tick_params(labelrotation=0)
df_service_calls.iloc[4:].plot(kind='bar', legend=True, ax=ax[1],color = colors)
ax[1].xaxis.set_tick_params(labelrotation=0)
ax[0].set(xlabel='')
ax[1].set(xlabel='')
ax[1].legend(['current','left'])
fig.suptitle('Frequency of customer service calls', fontsize=18);
ax[0].set(title='overall appearance')
ax[1].set(title='area should be adressed');

In [ ]:
# Line plot for findings in logistic regression
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots()
ax.plot(p_y,color='#7851A9')
ax.vlines(x=p_y[p_y>=0.5].index[0], ymin=0, ymax=1, colors='#56A0D3');

# ann.remove
ann = ax.annotate(s='The threshold \n      value',
            xytext=[0.1,0.71],xy=[0.44,0.7],xycoords='axes fraction',
            fontsize=13,arrowprops=dict(facecolor='black',linewidth=0.05))
ax.set(title='Logistic regression model');

# 6) Formulating recommendations

In [ ]:
## Insights and recommendations from the data set

# At first the company should pay attention for the top 4
# cities from which customers may leave 
# as they represent the biggest part of ones who may leave soon
##
# Furthermore, customers that use the international plan might leave as they are abroad and 
# it is uncertain if they are in travel and use the international package or moved fully.
# So, it is relevant to focus on them
##
# In addition company should emphasize on such customers who has often called the service
# as users might be not satisfied with the conditions and are also likely to leave sooner or 
#                                                                                  later
##
# The logistic regression in my case, unfortunately, did not give any hint, who the company 
# should call in their upcoming campaign
##
# In conclusion, the company should aim at:
# - customers who live in cities that are under observation
# - customers who use international plan
# - customers who often contacted the company service